In [ ]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
# from __future__ import unicode_literals
import pandas as pd

In [ ]:
# data_csv_number.to_csv('./data/train_data_number.csv', sep=',', index=False)
data_csv_number = pd.read_csv('./data_train_final.csv')
uni_ass = np.unique(data_csv_number['ASS_ASSIGNMENT'])
print data_csv_number.keys()
print uni_ass

In [ ]:
data_csv_number

In [ ]:
subdata = dict()
for ass in uni_ass:
    subdata[ass] = data_csv_number[data_csv_number['ASS_ASSIGNMENT'] == ass]

In [ ]:
subdata['CMS'].drop(['ASS_ASSIGNMENT','prediction', 'DATE', 'hour', 'minute'], axis=1)

In [ ]:
subdata['Téléphonie']['prediction'].values

In [ ]:
#Create training data 
X = dict()
Y = dict()

for ass in uni_ass:
    X[ass] = subdata[ass].drop(['ASS_ASSIGNMENT','prediction', 'DATE', 'hour', 'minute'], axis=1).values
    Y[ass] = subdata[ass]['prediction'].values

# Tuning params

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.decomposition import PCA                                            
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV

In [ ]:
def linExLoss(y, y_pred):
    result = 0;
    alpha = 0.1
    for yt,yp in zip(y,y_pred):
        tehta = alpha*(yt-yp)
        result += np.exp(tehta)-tehta-1
    return result/len(y)

def newScore(y, y_pred):
    return 1/linExLoss(y, y_pred)

In [ ]:
best_params = []

for ass in uni_ass:
    
    print ("++++++++++++ ",ass)
#     model_p = SVR(C=2.0, epsilon=0.2)
#     model_p = SVR(C=2.0, epsilon=0.4)
    
#     model_p =  GradientBoostingRegressor(              
#                     n_estimators=300,                              
#                     learning_rate=0.2,                            
#                     random_state=422)
#     model_p = Pipeline([                                      
#                     ('pca', PCA(n_components=self.n_components)),                    
#                     ('reg', GradientBoostingRegressor(                               
#                         n_estimators=300,                              
#                         learning_rate=0.2,                            
#                         random_state=42))                                            
#                 ])
#     model_p = RandomForestRegressor(n_estimators=300, max_depth=3, random_state=2)

    rfc = RandomForestRegressor(random_state=23)
    param_grid = { 
        'n_estimators' : range(100,300,50),#[50, 100, 150, 200],
        'max_depth' : [1,2,3]
    }
    model_p = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, n_jobs=6, scoring=newScore)
    model_p.fit(X[ass], Y[ass])
    best_params.append(model_p.best_params_)
    print model_p
    print model_p.best_score_
    print model_p.best_params_